## Dans ce notebook nous allons générer des données pour notre table orders

In [1]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string
from unidecode import unidecode
import warnings 
warnings.filterwarnings("ignore")

pd.set_option("display.max_rows", None)

In [2]:
# lst des noms de colonne de notre dataframe orders
lst_columns = ["Order_ID", "Cust_ID", "Shipping_cost_ID", "Number_items", "delivery_address", "Order_date", "Shipped_date", "Delivery_date", "Delivery_formula", "Payment_channel", "Total_amount", "Total_amount_invoiced"]

In [3]:
# chargement des dataframe dont nous 
df_cust = pd.read_csv("dataset/customers.csv")
df_product = pd.read_csv("dataset/products.csv")
df_geo = pd.read_csv("dataset/geographies.csv")
df_shipping_cost = pd.read_csv("dataset/shipping_cost.csv")

In [4]:
def district_dep (df_geo, department, df_shipping_cost):
    districts = df_geo[df_geo["Department"] == department]["District"]
    districts = districts.tolist()
    district_shipping = df_shipping_cost["District"]
    district_shipping = district_shipping.tolist()
    
    districts_commune = [x for x in district_shipping if x in districts]
    if not districts_commune :
        districts_commune = [random.choice(district_shipping)]
        
    return districts_commune # une liste

In [5]:
def info_shipping(df_shipping_cost, Cust_department , lst_district, formule) :
    info_shipping = []
    lst_district_ship = df_shipping_cost["District"]
    lst_district_ship = lst_district_ship.tolist()
    # choix aleatoire
    random_district = random.choice(lst_district)
    while (random_district not in lst_district_ship):
        random_district = random.choice(lst_district)
        
    shipping_cost_id = df_shipping_cost[df_shipping_cost["District"] == random_district]["Shipping_cost_ID"]
    shipping_cost_id = shipping_cost_id.tolist()
    shipping_cost_id = shipping_cost_id[0]    
      
        
    if (formule =="Standard"):
        delivery_cost = df_shipping_cost[df_shipping_cost["Shipping_cost_ID"] == shipping_cost_id]["Delivery_fee_standard"]
    if (formule =="Express"):
        delivery_cost = df_shipping_cost[df_shipping_cost["Shipping_cost_ID"] == shipping_cost_id]["Delivery_fee_express"]
    
    delivery_cost = delivery_cost.tolist()
    delivery_cost = delivery_cost[0]
    
    info_shipping.append(random_district)
    info_shipping.append(shipping_cost_id)
    info_shipping.append(delivery_cost)
    
    return info_shipping  # adress livraison, cost_id, delivery_cost

In [6]:
def generate_random_order_date(start_date, end_date):
    # conversion des dates en objet datetime
    if not isinstance(start_date, datetime):
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if not isinstance(end_date, datetime):
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # nous allons calculer le nombre de jour entre les deux dates
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_date = start_date + timedelta(days=random_days)
    random_date = random_date.strftime("%Y-%m-%d")
    
    return random_date

def generate_random_date(date_receive, day): # shipped_date et delivery_date
    # conversion des dates en objet datetime
    if not isinstance(date_receive, datetime):
        date_receive = datetime.strptime(date_receive, "%Y-%m-%d")
    
    shipped_date = date_receive + timedelta(days=day)
    shipped_date = shipped_date.strftime("%Y-%m-%d")
    
    return shipped_date

In [7]:
def range_orders (count):
    lst_order = []
    lst_payment_channel = ["Orange-Money", "Wave", "Free-Money"]
    lst_delivery_formula = ["Standard", "Express"]

    for i in range(count):
        dic_orders = {}
        customer = df_cust.sample()
        cust_date = customer["Cust_date_register"].iloc[0]
        customer_departement = customer["Cust_department"].iloc[0]
        district = district_dep(df_geo, customer_departement, df_shipping_cost) # une liste
        formula = random.choice(lst_delivery_formula)
        info_ship = info_shipping(df_shipping_cost, customer_departement , district, formula) # address livraison, cost_id, delivery_cost

        dic_orders["Cust_ID"] = customer["Cust_ID"].iloc[0]
        dic_orders["Shipping_cost_ID"] = info_ship[1]
        dic_orders["Number_items"] = random.choice([4,3,2,1,5])
        dic_orders["delivery_address"] = info_ship[0]
        order_date = generate_random_order_date(cust_date, "2024-03-30")
        dic_orders["Order_date"] = order_date

        if(formula== "Standard"):
            dic_orders["Shipped_date"] = generate_random_date(order_date,  random.choice([3,4,5]) )
            dic_orders["Delivery_date"] = generate_random_date(dic_orders["Shipped_date"],  random.choice([2,3,4]) )
        if(formula== "Express"):
            dic_orders["Shipped_date"] = generate_random_date(order_date,  random.choice([1,2]) )
            dic_orders["Delivery_date"] = generate_random_date(dic_orders["Shipped_date"],  random.choice([0,1,2]) )

        dic_orders["Delivery_formula"] = formula
        dic_orders["Payment_channel"] = random.choice(lst_payment_channel)
        dic_orders["Total_amount"] = info_ship[2]
        dic_orders["Total_amount_invoiced"] =""
        lst_order.append(dic_orders)
        
    return lst_order

In [8]:
lst_order = range_orders(1500)

In [9]:
df_orders = pd.DataFrame(columns= lst_columns[1:])
for o in lst_order :
    df_orders = df_orders.append(o, ignore_index=True)
df_orders = df_orders.sort_values(by='Order_date', ascending=True)
df_orders = df_orders.reset_index(drop=True)

In [10]:
df_orders.head(5)

,Cust_ID,Shipping_cost_ID,Number_items,delivery_address,Order_date,Shipped_date,Delivery_date,Delivery_formula,Payment_channel,Total_amount,Total_amount_invoiced
0,A000021,Fee_070,3,Thiomby,2022-04-17,2022-04-22,2022-04-26,Standard,Orange-Money,3000,
1,A000025,Fee_004,5,Medina,2022-04-21,2022-04-25,2022-04-28,Standard,Wave,3000,
2,A000040,Fee_017,5,Grand Yoff,2022-04-25,2022-04-28,2022-05-02,Standard,Orange-Money,2000,
3,A000040,Fee_012,1,Sicap Liberte,2022-05-01,2022-05-02,2022-05-04,Express,Orange-Money,4500,
4,A000052,Fee_077,5,Sessene,2022-05-22,2022-05-24,2022-05-26,Express,Wave,3000,


In [11]:
list_Order_ID = []
for i in range(1, len(df_orders)+1):
    order_ID = f"O{i:06d}"
    list_Order_ID.append(order_ID)
    
df_orders["Order_ID"] = list_Order_ID

In [12]:
df_orders

,Cust_ID,Shipping_cost_ID,Number_items,delivery_address,Order_date,Shipped_date,Delivery_date,Delivery_formula,Payment_channel,Total_amount,Total_amount_invoiced,Order_ID
0,A000021,Fee_070,3,Thiomby,2022-04-17,2022-04-22,2022-04-26,Standard,Orange-Money,3000,,O000001
1,A000025,Fee_004,5,Medina,2022-04-21,2022-04-25,2022-04-28,Standard,Wave,3000,,O000002
2,A000040,Fee_017,5,Grand Yoff,2022-04-25,2022-04-28,2022-05-02,Standard,Orange-Money,2000,,O000003
3,A000040,Fee_012,1,Sicap Liberte,2022-05-01,2022-05-02,2022-05-04,Express,Orange-Money,4500,,O000004
4,A000052,Fee_077,5,Sessene,2022-05-22,2022-05-24,2022-05-26,Express,Wave,3000,,O000005
5,A000071,Fee_021,3,Medina Gounass,2022-05-30,2022-06-04,2022-06-08,Standard,Orange-Money,3000,,O000006
6,A000049,Fee_040,5,Tivavouane Diaksao,2022-06-02,2022-06-04,2022-06-06,Express,Orange-Money,4500,,O000007
7,A000020,Fee_086,4,Thienaba,2022-06-05,2022-06-06,2022-06-06,Express,Free-Money,3000,,O000008
8,A000079,Fee_015,5,Ngor,2022-06-07,2022-06-09,2022-06-10,Express,Wave,4500,,O000009
9,A000021,Fee_070,1,Thiomby,2022-06-19,2022-06-21,2022-06-23,Express,Orange-Money,4500,,O000010


In [13]:
# enregistrement du dataframe dans un fichier csv
df_orders.to_csv("dataset/orders.csv",encoding='utf-8-sig', index=False)

In [14]:
len(df_orders)

1500